In [1]:
import numpy as np
import matplotlib.pyplot as plt
import wave_bwf_rf64 
import contextlib
import math
from scipy import signal

In [2]:
filename = "1090.wav"
with contextlib.closing(wave_bwf_rf64.open(filename,'rb')) as f:
    fs = f.getframerate()
    num_frames = f.getnframes()
    iq_samples = f.readframes(num_frames)
iq_samples = np.frombuffer(iq_samples, dtype=np.int16) 
I = iq_samples[0::2]
Q = iq_samples[1::2]
iq_samples = I + 1j * Q
iq_samples = np.abs(iq_samples)
#len(iq_samples)

In [3]:
up = 5
down = 3
iq_samples = signal.resample_poly(iq_samples,up = up,down=down)
#len(iq_samples)

In [4]:
fs = fs*up/down
Ts = 1/fs
length = num_frames*Ts
#--------PREAMBLE--------#
a1 = 0
a2 = 0.5*10**-6
b1 = 1*10**-6
b2 = 1.5*10**-6
c1 = 3.5*10**-6
c2 = 4*10**-6
d1 = 4.5*10**-6
d2 = 5*10**-6
#------------------------#
#-----------DF-----------##     10 01 01 01 10       #
e1 = 8*10**-6
e2 = 8.5*10**-6
f1 = 9.5*10**-6
f2 = 10*10**-6
g1 = 10.5*10**-6
g2 = 11*10**-6
h1 = 11.5*10**-6
h2 = 12.5*10**-6
#------------------------#
preamble_df = np.arange(0*10**-6,13*10**-6,Ts)
print(Ts)
for i in range(len(preamble_df)):
    if  ((a1 <= preamble_df[i] < a2) or (d1 <= preamble_df[i] < d2) or (c1 <= preamble_df[i] < c2) or (b1 <= preamble_df[i] < b2)) or ((e1 <= preamble_df[i] < e2) or (f1 <= preamble_df[i] < f2) or (g1 <= preamble_df[i] < g2) or (h1 <= preamble_df[i] < h2)):
        preamble_df[i] = 1
    else:
        preamble_df[i] = 0
time_preamble = np.arange(0,len(preamble_df)*Ts,Ts)
#plt.plot(preamble_df,marker = "o")
#plt.show
#print(len(preamble_df))

2.5e-07


In [5]:
# minta keresése
arg = list()
for i in range(len(iq_samples)-len(preamble_df)-1):
# az 1 esek nagyobbak mint a 0-k
    if (iq_samples[i] > iq_samples[i+2] and
        iq_samples[i+1] > iq_samples[i+2] and
        iq_samples[i+3] < iq_samples[i+4] and
        iq_samples[i+5] > iq_samples[i+3] and

        iq_samples[i+14] > iq_samples[i+13] and
        iq_samples[i+15] > iq_samples[i+13] and
        iq_samples[i+17] < iq_samples[i+19] and
        iq_samples[i+17] < iq_samples[i+18] and  

        iq_samples[i+32] > iq_samples[i+31] and
        iq_samples[i+33] > iq_samples[i+31] and

        iq_samples[i+38] > iq_samples[i+37] and
        iq_samples[i+39] > iq_samples[i+37] and

        iq_samples[i+42] > iq_samples[i+41] and
        iq_samples[i+43] > iq_samples[i+41] and

        iq_samples[i+49] > iq_samples[i+45] and
        iq_samples[i+46] > iq_samples[i+45]):
            arg.append(i)
len(arg)

56309

In [6]:
# ki kellene listázni az összes 0 hibás üzenetet
# a minták az iq_samples[arg[i]:arg[i]+len(preamble_df)]
from crccheck.crc import Crc
messages = list()
raw_messages = list()
for i in arg:
    start = i
    stop1 = start + math.floor(8*10**-6/Ts)
    PREAMBLE = iq_samples[start:stop1]
    #----------------DF mintái------------------#
    stop2 = stop1 + math.floor(5*10**-6/Ts)
    DF = iq_samples[stop1:stop2]
    #----------------CA mintái------------------#
    stop3 = stop2 + math.floor(3*10**-6/Ts)
    CA = iq_samples[stop2:stop3]
    #----------------ICAO mintái------------------#
    stop4 = stop3 + math.floor(24*10**-6/Ts)
    ICAO = iq_samples[stop3:stop4]
    #----------------ME mintái------------------#
    stop5 = stop4 + math.floor(56*10**-6/Ts)
    ME = iq_samples[stop4:stop5]
    #----------------PI mintái------------------#
    stop6 = stop5 + math.floor(24*10**-6/Ts)
    PI = iq_samples[stop5:stop6]
    full = np.concatenate([PREAMBLE,DF,CA,ICAO,ME,PI])
    
    bits = np.zeros(112,dtype = np.int64)
    full_bitek = full
    preamble_bits = full_bitek[:16*2]
    full_bitek = full_bitek[16*2:]
    raw_mess = list()
    #-------------------------4 MSPS-------------------------------#
    for n in range(112):
        raw_mess.append(full_bitek[4*n:4*n+4])
        if (sum(full_bitek[4*n:4*n+2]) < sum(full_bitek[4*n+2:4*n+4])):
            bits[n] = 0
        elif (sum(full_bitek[4*n:4*n+2]) > sum(full_bitek[4*n+2:4*n+4])):
            bits[n] = 1
        else:
            bits[n] = -1

    df_bits = bits[:5]
    #df_bits = ''.join(str(b) for b in df_bits.astype(int))
    ca_bits = bits[5:8]
    #ca_bits = ''.join(str(b) for b in ca_bits.astype(int))
    icao_bits = bits[8:32]
    #icao_bits = ''.join(str(b) for b in icao_bits.astype(int))
    me_bits = bits[32:88]
    pi_bits = bits[88:]
    type_code_bits = me_bits[:5]
    type_code_bits = ''.join(str(b) for b in type_code_bits.astype(int))
    #print(df_bits,ca_bits,icao_bits,me_bits,pi_bits,type_code_bits)
    ADS_B = Crc(width=24,poly=0x1FFF409,initvalue=0,xor_output=0,reflect_input=False,reflect_output=False)
    df_bits_string = ''.join(str(b) for b in df_bits)
    ca_bits_string = ''.join(str(b) for b in ca_bits)
    icao_bits_string = ''.join(str(b) for b in icao_bits)
    me_bits_string = ''.join(str(b) for b in me_bits)
    pi_bits_string = ''.join(str(b) for b in pi_bits)
    data = df_bits_string+ca_bits_string+icao_bits_string+me_bits_string+pi_bits_string
    data = hex(int(data, 2))[2:]
    if len(data) % 2 != 0:
        data = "0" + data
    
     #print(data)
    data_ = bytes.fromhex(data)    
    err_mess = list()
    remainder = ADS_B.calc(data_)
    if (remainder == 0):
        messages.append(data)
        raw_messages.append(raw_mess)
    else:
        err_mess.append(data)

print(len(messages),len(err_mess))

43 1


In [12]:
%load_ext autoreload
%autoreload 2
from functions import decoder
from lut import message_velocity_lut as mv
for i in messages:
    typecode,cap,icao,mess = decoder(i)
    if typecode == 19:
        print(i,typecode,icao,mess)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
8d4d2278990939a938043bf0b993 19 4d2278 ('sebesseg: 452.6897392254435knots', 'szog: 136.43209618416466 fok', 'vertikalis sebesseg: 0ft/min', 'subsonic', 'ground speed')
8d4d2278990939a938043bf0b993 19 4d2278 ('sebesseg: 452.6897392254435knots', 'szog: 136.43209618416466 fok', 'vertikalis sebesseg: 0ft/min', 'subsonic', 'ground speed')
8d4d2278990939a938043bf0b993 19 4d2278 ('sebesseg: 452.6897392254435knots', 'szog: 136.43209618416466 fok', 'vertikalis sebesseg: 0ft/min', 'subsonic', 'ground speed')
8d06a30d9915961ff0043d4c35ff 19 6a30d ('sebesseg: 478.05961971285546knots', 'szog: 302.09431228447545 fok', 'vertikalis sebesseg: 0ft/min', 'subsonic', 'ground speed')
8d49d3619910dc95700436cba927 19 49d361 ('sebesseg: 277.2381647609146knots', 'szog: 127.82060614811454 fok', 'vertikalis sebesseg: 0ft/min', 'subsonic', 'ground speed')
8d4b9e01990d552438043905a070 19 4b9e01 ('sebesseg: 445.58276447816064kno